## all-atom diffusion map

### setup

In [ ]:
import os
import matplotlib as mpl
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import sys
import math
import time
import pickle
from copy import deepcopy

import numpy as np

import sklearn.preprocessing as pre
from sklearn.model_selection import train_test_split
from sklearn import manifold

from scipy.linalg import orthogonal_procrustes
from scipy.linalg import eigh

from collections import OrderedDict

import mdtraj as md
import nglview as nv

In [ ]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2'''

In [ ]:
# fix random seed for reproducibility
randstate = 200184
np.random.seed(randstate)

### loading simulation trajectory

In [ ]:
trj_dir = "../C24/"
pdb_file = os.path.join(trj_dir, "after_pr_protein.pdb")
trj_file = os.path.join(trj_dir, "md_protein_fit.xtc")

In [ ]:
# loading
traj = md.load(trj_file, top=pdb_file)

In [ ]:
# rot+trans alignment of trajectory to first frame
traj.center_coordinates(mass_weighted=False)
traj.superpose(traj[0])

In [ ]:
print('How many frames? %s' % traj.n_frames)
print('How many atoms?    %s' % traj.n_atoms)
print(traj.xyz.shape)

In [ ]:
# visualizing
view = nv.show_mdtraj(traj)
view.remove_cartoon()
view.add_ball_and_stick()
view

### loading principal moments of gyration tensor

In [ ]:
npzfile = np.load('../0_traj2pdh2t/princMom.npz')
sorted(npzfile.files)
princMom = npzfile['princMom']
Rg = npzfile['Rg']

print(princMom.shape)
print(Rg.shape)

### loading h2t

In [ ]:
npzfile = np.load('../0_traj2pdh2t/h2t.npz')
sorted(npzfile.files)
h2t = npzfile['h2t']

print(len(h2t))

### subsampling trajectory to manageable size for dMaps

In [ ]:
startFrame = 0
stopFrame = 100000
skipper = 10
traj_ss = deepcopy(traj[startFrame:stopFrame:skipper])

print('How many frames? %s' % traj_ss.n_frames)
print('How many atoms?    %s' % traj_ss.n_atoms)
print(traj_ss.xyz.shape)

princMom_ss = princMom[startFrame:stopFrame:skipper]
Rg_ss = Rg[startFrame:stopFrame:skipper]
h2t_ss = h2t[startFrame:stopFrame:skipper]

### generating h2t flipped and mirror inverted subsampled trajectories

In [ ]:
traj_ss_flip = deepcopy(traj_ss)
for i in range(traj_ss_flip.n_frames):
    traj_ss_flip.xyz[i] = np.flipud(traj_ss_flip.xyz[i])

In [ ]:
traj_ss_mir = deepcopy(traj_ss)
for i in range(traj_ss_mir.n_frames):
    xyz = deepcopy(traj_ss_mir.xyz[i])
    xyz[:,0] = -xyz[:,0]
    traj_ss_mir.xyz[i] = xyz
print(xyz)

### computing pairwise RMSD between all frames of traj_ss

rot + trans minimized

symmetrized (minimized) mirror inversion (and h2t flip for molecules symmetric about their central axis)

In [ ]:
N = traj_ss.n_frames

P = np.zeros((N,N),dtype=float)
for i in range(N-1):
    RMSD_i_normal = md.rmsd(traj_ss[i+1:],traj_ss[i],frame=0)
    RMSD_i_flip = md.rmsd(traj_ss_flip[i+1:],traj_ss[i],frame=0)
    RMSD_i_mir = md.rmsd(traj_ss_mir[i+1:],traj_ss[i],frame=0)
    RMSD_i = np.minimum(np.minimum(RMSD_i_normal,RMSD_i_flip),RMSD_i_mir)
    P[i,i+1:] = RMSD_i
P = P + P.T

outfile='P.npz'
np.savez(outfile, P=P)

### epsilon scan

In [ ]:
# scanning for appropriate gaussian kernal width
log_eps_array = np.arange(-8,1,1)
sumA = np.empty_like(log_eps_array)

for i in range(len(log_eps_array)):
    eps = np.exp(log_eps_array[i])
    sumA[i] = np.sum(np.exp(-P**2/(2*eps)),axis=None)

In [ ]:
# plotting
fig, ax = plt.subplots()
ax.scatter(log_eps_array, np.log(sumA))
ax.plot([log_eps_array[0],log_eps_array[-1]],[np.log(N**2),np.log(N**2)],'k:')
ax.plot([log_eps_array[0],log_eps_array[-1]],[np.log(N),np.log(N)],'k:')
ax.set_xlabel("log(eps)")
ax.set_ylabel("log(sumA)")
if showPlots:
    plt.draw()
    plt.show()

In [ ]:
# selecting
eps = np.exp(-3)

### running diffusion maps

In [ ]:
# dMaps

outfile='P.npz'
npzfile = np.load(outfile)
sorted(npzfile.files)
P = npzfile['P']

P = np.exp(-P**2/(2*eps))
D = np.sum(P,axis=1)
P = np.matmul(np.matmul(np.diag(D**(-0.5)),P),np.diag(D**(-0.5))) # constructing Ms as symmetric matrix for diagonalization


nEvals=20
lamb, psi = eigh(P,eigvals=(P.shape[0]-nEvals,P.shape[0]-1)) # scipy eigh to specify only computation of leading evals
psi = np.matmul(np.diag(D**(-0.5)),psi) # converting evecs of Ms to evecs of M; M and Ms share evals

idx_sort = np.flip(np.argsort(lamb))

lamb = lamb[idx_sort]
psi = psi[:,idx_sort]

In [ ]:
# plotting eval spectrum
fig = plt.figure()
ax = fig.add_subplot(111)
ax.bar(np.arange(2,len(lamb)+1), lamb[1:])
ax.set_xlabel('eval idx')
ax.set_ylabel('eval')
plt.xticks(np.arange(2, len(lamb)+1, step=1))
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evals.png', dpi=300)
plt.close()

In [ ]:
# plotting embedding

plot_stride = 1

colorMap = princMom_ss[::plot_stride,2]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(psi[::plot_stride,1], psi[::plot_stride,2], c=colorMap)
ax.set_xlabel('psi_2')
ax.set_ylabel('psi_3')
plt.xlim([np.min(psi[:,1])*1.1,np.max(psi[:,1])*1.1])
plt.ylim([np.min(psi[:,2])*1.1,np.max(psi[:,2])*1.1])
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evecs_23.png', dpi=300)
plt.close()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(psi[::plot_stride,1], psi[::plot_stride,2], psi[::plot_stride,3], c=colorMap)
ax.set_xlabel('psi_2')
ax.set_ylabel('psi_3')
ax.set_zlabel('psi_4')
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evecs_234.png', dpi=300)
plt.close()

### Nystrom extension

embedding all remaining points

In [ ]:
# Nystrom extension applied to all frames in data set

z = []

start = time.time()

print('Commencing Nystrom extension...')
for i in range(traj.n_frames):
    RMSD_i_normal = md.rmsd(traj_ss,traj[i],frame=0)
    RMSD_i_flip = md.rmsd(traj_ss_flip,traj[i],frame=0)
    RMSD_i_mir = md.rmsd(traj_ss_mir,traj[i],frame=0)
    RMSD_i = np.minimum(np.minimum(RMSD_i_normal,RMSD_i_flip),RMSD_i_mir)

    RMSD_i = np.exp(-RMSD_i**2/(2*eps))

    RMSD_i /= np.sum(RMSD_i)

    psi_Nystrom = np.divide( np.matmul(RMSD_i,psi), lamb)
    
    z.append(psi_Nystrom)
    
    if np.mod(i+1,10) == 0:
        print('\tNystromed %d of %d frames...' % (i+1,traj.n_frames))

z = np.array(z)
z = z.reshape(z.shape[0],-1)

print('DONE!')
print('')

end = time.time()
print("Elapsed time %.2f (s)" % (end - start))

### saving data

In [ ]:
# saving dMap embedding of all data
np.savez('data.npz', z=z)

In [ ]:
# plotting embedding w/ Nystrom

plot_stride = 100

colorMap = princMom[::plot_stride,0]

fig = plt.figure()
ax = fig.add_subplot(111)
ax.scatter(z[::plot_stride,1], z[::plot_stride,2], c=colorMap)
ax.set_xlabel('psi_2')
ax.set_ylabel('psi_3')
plt.xlim([np.min(z[:,1])*1.1,np.max(z[:,1])*1.1])
plt.ylim([np.min(z[:,2])*1.1,np.max(z[:,2])*1.1])
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evecs_23_Nystrom.png', dpi=300)
plt.close()

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
ax.scatter(z[::plot_stride,1], z[::plot_stride,2], z[::plot_stride,3], c=colorMap)
ax.set_xlabel('psi_2')
ax.set_ylabel('psi_3')
ax.set_zlabel('psi_4')
if showPlots:
    plt.draw()
    plt.show()
fig.savefig('evecs_234_Nystrom.png', dpi=300)
plt.close()